# core

> Progress for week of 09/10

In [ ]:
#| default_exp core

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

23/09/18 14:06:32 WARN Utils: Your hostname, Tesfas-MacBook-Pro-738.local resolves to a loopback address: 127.0.0.1; using 172.28.233.209 instead (on interface en0)
23/09/18 14:06:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/18 14:06:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



# Milestone 1

Below I demonstrate what I believe to be are at least 4 different operations (collect, filter, groupby, avg, and applyInPandas) on one dataset.

In [ ]:
df.collect()

[Row(color='red', fruit='banana', v1=1, v2=10),
 Row(color='blue', fruit='banana', v1=2, v2=20),
 Row(color='red', fruit='carrot', v1=3, v2=30),
 Row(color='blue', fruit='grape', v1=4, v2=40),
 Row(color='red', fruit='carrot', v1=5, v2=50),
 Row(color='black', fruit='carrot', v1=6, v2=60),
 Row(color='red', fruit='banana', v1=7, v2=70),
 Row(color='red', fruit='grape', v1=8, v2=80)]

In [ ]:
df.filter(df.a == 1).show()

+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|  red|banana|  1| 10|
+-----+------+---+---+



In [ ]:
df = spark.createDataFrame([
    ['red', 'banana', 1, 10], ['blue', 'banana', 2, 20], ['red', 'carrot', 3, 30],
    ['blue', 'grape', 4, 40], ['red', 'carrot', 5, 50], ['black', 'carrot', 6, 60],
    ['red', 'banana', 7, 70], ['red', 'grape', 8, 80]], schema=['color', 'fruit', 'v1', 'v2'])
df.show()

In [ ]:
df.groupby('color').avg().show()

+-----+-------+-------+
|color|avg(v1)|avg(v2)|
+-----+-------+-------+
|  red|    4.8|   48.0|
| blue|    3.0|   30.0|
|black|    6.0|   60.0|
+-----+-------+-------+



In [ ]:
# def plus_mean(pandas_df):
#     return pandas_df.assign(v2=pandas_df.v2 - pandas_df.v2.mean())

# df.groupby('color').applyInPandas(plus_mean, schema=df.schema).show()

/opt/homebrew/Caskroom/miniforge/base/envs/sparkenv/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead


+-----+------+---+---+
|color| fruit| v1| v2|
+-----+------+---+---+
|black|carrot|  6|  0|
| blue|banana|  2|-10|
| blue| grape|  4| 10|
|  red|banana|  1|-38|
|  red|carrot|  3|-18|
|  red|carrot|  5|  2|
|  red|banana|  7| 22|
|  red| grape|  8| 32|
+-----+------+---+---+



In [ ]:
# /opt/homebrew/Caskroom/miniforge/base/envs/sparkenv/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/sql/pandas/serializers.py:229: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead

# +-----+------+---+---+
# |color| fruit| v1| v2|
# +-----+------+---+---+
# |black|carrot|  6|  0|
# | blue|banana|  2|-10|
# | blue| grape|  4| 10|
# |  red|banana|  1|-38|
# |  red|carrot|  3|-18|
# |  red|carrot|  5|  2|
# |  red|banana|  7| 22|
# |  red| grape|  8| 32|
# +-----+------+---+---+


SyntaxError: invalid syntax (1814594154.py, line 1)

Here I begin to think about manipulating key Spark configurations such as the number of partitions.

In [ ]:
initialNumPartitions = df.rdd.getNumPartitions()
df2 = df.repartition(numPartitions=3)
manipulatedNumPartitions = df2.rdd.getNumPartitions()
print(initialNumPartitions, manipulatedNumPartitions)